# **Environment Setup and Imports**

In [ ]:
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

# **Data Preprocessing**


## *Data Loading*

In [ ]:
# --- Data Loading & Alignment ---
with open('rnn_features.pkl','rb') as f:
    rnn = pickle.load(f)
with open('cnn_features.pkl','rb') as f:
    cnn = pickle.load(f)

# Build DataFrames
df_rnn = pd.DataFrame(rnn['features'], index=rnn['ids'])
df_rnn['label'] = rnn['labels']
df_cnn = pd.DataFrame(cnn['features'], index=cnn['ids'])
df_cnn['label'] = cnn['labels']

# Align on common IDs and sync labels
common = df_rnn.index.intersection(df_cnn.index)
df_rnn = df_rnn.loc[common]
df_cnn = df_cnn.loc[common]
mismatch = df_rnn['label'] != df_cnn['label']
if mismatch.any():
    df_rnn.loc[mismatch, 'label'] = df_cnn.loc[mismatch, 'label']
assert (df_rnn['label'] == df_cnn['label']).all(), "Label sync failed"

# Prepare fused features and labels
X = np.concatenate([
    df_cnn.drop(columns='label').values,
    df_rnn.drop(columns='label').values
], axis=1)
y = df_cnn['label'].values
print("Fused feature shape:", X.shape, "Label distribution:", Counter(y))

FileNotFoundError: [Errno 2] No such file or directory: 'rnn_features.pkl'

## *Data Splitting*

In [ ]:
# --- Train/Validation/Test Split ---
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
# Second split: try stratified, fallback to unstratified if too few samples
try:
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
    )
except ValueError:
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, random_state=42
    )
print("Split sizes:", Counter(y_train), Counter(y_val), Counter(y_test))

Split sizes: Counter({np.float32(0.0): 66, np.float32(1.0): 4}) Counter({np.float32(0.0): 15}) Counter({np.float32(0.0): 14, np.float32(1.0): 1})


In [ ]:
# FIRST SPLIT (train vs temp)
#strat1 = y if len(set(y))>1 else None
#X_train, X_temp, y_train, y_temp = train_test_split(
#    X, y,
#    test_size=0.30,
#    random_state=42,
#    stratify=strat1
#)
#
## SECOND SPLIT (val vs test) with pre-check
#counts_temp = Counter(y_temp)
## if any class has <2 samples, drop stratify
#if any(c<2 for c in counts_temp.values()):
#    print("Too few examples in one class for stratified split, splitting WITHOUT stratify")
#    strat2 = None
#else:
#    strat2 = y_temp
#
#X_val, X_test, y_val, y_test = train_test_split(
#    X_temp, y_temp,
#    test_size=0.50,
#    random_state=42,
#    stratify=strat2
#)
#
#print("\nSplit sizes:")
#print(" Train:", X_train.shape, Counter(y_train))
#print(" Val:  ", X_val.shape,   Counter(y_val))
#print(" Test: ", X_test.shape,  Counter(y_test))

# **Fusion Model**

## *Model Building*

In [ ]:
# --- Baseline Fusion Model ---
model = Sequential([
    tf.keras.Input(shape=(X.shape[1],)),
    Dense(128, activation='relu'), Dropout(0.3),
    Dense(64, activation='relu'),  Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)
model.summary()

Model: "sequential_338"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1014 (Dense)              │ (None, 128)            │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_676 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1015 (Dense)              │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_677 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1016 (Dense)              │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,929 (113.00 KB)

 Trainable params: 28,929 (113.00 KB)

 Non-trainable params: 0 (0.00 B)

## *Model Training*

In [ ]:
# --- Training ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50, batch_size=32, verbose=2
)

## *Model Evaluation*

In [ ]:
# --- Evaluation on Test Set ---
y_test_prob = model.predict(X_test).ravel()
y_test_pred = (y_test_prob >= 0.5).astype(int)
print("Confusion matrix on test set:")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_test_prob))

# --- Stratified K‑Fold Cross‑Validation ---
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []
for train_idx, test_idx in skf.split(X, y):
    X_tr, X_te = X[train_idx], X[test_idx]
    y_tr, y_te = y[train_idx], y[test_idx]
    m = Sequential([
        tf.keras.Input(shape=(X.shape[1],)),
        Dense(128, activation='relu'), Dropout(0.3),
        Dense(64, activation='relu'),  Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    m.compile(optimizer='adam', loss='binary_crossentropy', metrics=[])
    m.fit(X_tr, y_tr, epochs=30, batch_size=32, verbose=0)
    prob = m.predict(X_te).ravel()
    auc_scores.append(roc_auc_score(y_te, prob))
print("Stratified K-Fold mean ROC AUC:", np.mean(auc_scores))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Confusion matrix on test set:
[[14  0]
 [ 1  0]]
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97        14
         1.0       0.00      0.00      0.00         1

    accuracy                           0.93        15
   macro avg       0.47      0.50      0.48        15
weighted avg       0.87      0.93      0.90        15

Test ROC AUC: 0.7142857142857143


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Stratified K-Fold mean ROC AUC: 0.5684210526315789
